### Import modules

In [2]:
import os
import numpy as np
import pandas as pd
import time
import tensorflow as tf
from sklearn import preprocessing
from tensorflow import keras
import os
os.getcwd()
os.chdir('C:/Users/ASUS/Desktop/Github/Face_Recognition_with_mask')
os.getcwd()

'C:\\Users\\ASUS\\Desktop\\Github\\Face_Recognition_with_mask'

### Load Test data

In [3]:
age_label_list = ['0-9','10-19','20-29','30-39','40-49','50-59','60-69','70-79','80-89','90-100']
# example of datasets name
# Test_dataset_50id_vector_Non_masked.txt
# Test_dataset_50id_vector_Masked_Ver_2.txt
file = open("C:/Users/ASUS/Desktop/Github/Face_Recognition_with_mask/Data/Test_dataset_50id_vector_Masked_Ver_2.txt",'r')
lines = file.readlines()
first = True
df = None
count = 1
vector_list = []
gender_list = []
race_list = []
age_list = []
id_list = []
for line in lines:
    data = line.split('|')
    image_id = line.split('/')[3]
    image_id = image_id[:image_id.find('\\')]
    id_list.append(image_id)
    list_of_vector = data[1].split(' ')
    list_of_vector = map(float, list_of_vector)
    list_of_vector = list(list_of_vector)
    vector_list.append(list_of_vector)
    list_age = data[2].split(' ')
    list_gender = data[3].split(' ')
    list_race = data[4].split(' ') 
    if np.argmax(list_gender) == 0:
        gender_list.append('Women')
    elif np.argmax(list_gender) == 1:
        gender_list.append('Men')   
    if np.argmax(list_race) == 0:
        race_list.append('Asian')
    elif np.argmax(list_race) == 1:
        race_list.append('Indian')
    elif np.argmax(list_race) == 2:
        race_list.append('Black')
    elif np.argmax(list_race) == 3:
        race_list.append('White')
    elif np.argmax(list_race) == 4:
        race_list.append('Middle_Eastern')
    elif np.argmax(list_race) == 5:
        race_list.append('Latino_Hispanic') 
    list_age = map(float, list_age)
    list_age = list(list_age)
    age_predictions = np.array(list_age)
    output_indexes = np.array([i for i in range(0, 101)])
    apparent_age = np.sum(age_predictions * output_indexes)
    apparent_age = round(apparent_age)
    if apparent_age >= 0 and apparent_age <= 9:
        age_list.append('0-9')
    elif apparent_age >= 10 and apparent_age <= 19:
        age_list.append('10-19')
    elif apparent_age >= 20 and apparent_age <= 29:
        age_list.append('20-29')
    elif apparent_age >= 30 and apparent_age <= 39:
        age_list.append('30-39')
    elif apparent_age >= 40 and apparent_age <= 49:
        age_list.append('40-49')
    elif apparent_age >= 50 and apparent_age <= 59:
        age_list.append('50-59')
    elif apparent_age >= 60 and apparent_age <= 69:
        age_list.append('60-69')
    elif apparent_age >= 70 and apparent_age <= 79:
        age_list.append('70-79')
    elif apparent_age >= 80 and apparent_age <= 89:                   
        age_list.append('80-89')
    elif apparent_age >= 90 and apparent_age <= 100:
        age_list.append('90-100')

dataframe_test = pd.DataFrame(vector_list)
one_hot_gender = pd.get_dummies(gender_list)
one_hot_race = pd.get_dummies(race_list)
one_hot_age = pd.get_dummies(age_list)

for i in range(10):
    if age_label_list[i] not in one_hot_age.columns:
        one_hot_age.insert(loc=i,column=age_label_list[i],value = 0)

le = preprocessing.LabelEncoder()
le.fit(id_list)
encoded_id_test = le.transform(id_list)        

dataframe_test = dataframe_test.join(one_hot_age)
dataframe_test = dataframe_test.join(one_hot_gender)
dataframe_test = dataframe_test.join(one_hot_race)
dataframe_test['id'] = encoded_id_test
dataframe_test

,0,1,2,3,4,5,6,7,8,9,...,90-100,Men,Women,Asian,Black,Indian,Latino_Hispanic,Middle_Eastern,White,id
0,-0.005811,0.006799,0.006527,0.006695,0.014906,0.011761,0.002441,0.014137,0.001551,0.000994,...,0,1,0,0,0,0,0,0,1,0
1,-0.000615,0.004171,0.008273,0.014302,0.010149,0.014710,0.005147,0.013600,0.005837,0.008584,...,0,0,1,0,0,0,0,0,1,0
2,-0.004168,-0.001410,0.005084,0.008728,0.003481,0.017883,-0.002308,0.017414,0.006853,0.008259,...,0,0,1,0,0,0,0,0,1,0
3,0.000304,0.011040,0.014343,0.013360,0.009079,0.017290,0.014120,0.020558,-0.001548,0.008252,...,0,0,1,0,0,0,0,0,1,0
4,-0.003455,-0.003399,0.012119,0.010312,0.004979,0.024713,0.004397,0.020256,0.018522,0.009038,...,0,1,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2082,0.014161,0.008186,0.017251,0.013830,0.030316,-0.001384,0.008802,0.016736,-0.002087,0.004245,...,0,0,1,0,0,0,0,0,1,49
2083,0.010095,0.012177,0.013166,0.010713,0.036137,0.000886,0.002403,0.028041,0.012473,0.016539,...,0,1,0,0,0,1,0,0,0,49
2084,0.014688,0.009354,0.012795,0.011314,0.033331,0.011004,0.000947,0.019660,0.006582,0.028804,...,0,0,1,0,0,0,0,0,1,49
2085,0.018985,0.005582,0.009959,0.016992,0.031971,0.009151,-0.000118,0.028676,0.006455,0.012933,...,0,1,0,0,0,0,0,0,1,49


### Preprocessing Data

In [4]:
age_label_list = ['0-9','10-19','20-29','30-39','40-49','50-59','60-69','70-79','80-89','90-100']
gender_label_list = ['Men','Women']
race_label_list = ['Asian','Black','Indian','Latino_Hispanic','Middle_Eastern','White']

all_features_label_list = age_label_list + gender_label_list + race_label_list
age_gender_label = race_label_list
age_race_label = gender_label_list
gender_race_label = age_label_list
age_label = gender_label_list + race_label_list
gender_label = age_label_list + race_label_list
race_label = age_label_list + gender_label_list

all_feature_activate = False
if all_feature_activate is not True:
    dataframe_test = dataframe_test.drop(all_features_label_list, axis=1)

In [5]:
dataframe_test.head()

,0,1,2,3,4,5,6,7,8,9,...,2613,2614,2615,2616,2617,2618,2619,2620,2621,id
0,-0.005811,0.006799,0.006527,0.006695,0.014906,0.011761,0.002441,0.014137,0.001551,0.000994,...,-0.005224,-0.022632,-0.011859,0.019855,-0.008901,-0.008245,-0.014659,0.000927,0.017252,0
1,-0.000615,0.004171,0.008273,0.014302,0.010149,0.014710,0.005147,0.013600,0.005837,0.008584,...,-0.000316,-0.016601,-0.001300,0.022648,0.003575,0.014947,-0.019859,0.008109,0.021828,0
2,-0.004168,-0.001410,0.005084,0.008728,0.003481,0.017883,-0.002308,0.017414,0.006853,0.008259,...,-0.011242,-0.016792,-0.007640,0.021043,-0.004024,0.000192,-0.012531,0.004651,0.023042,0
3,0.000304,0.011040,0.014343,0.013360,0.009079,0.017290,0.014120,0.020558,-0.001548,0.008252,...,-0.012067,-0.018902,-0.004995,0.020847,-0.003581,-0.001372,-0.016296,0.007014,0.029171,0
4,-0.003455,-0.003399,0.012119,0.010312,0.004979,0.024713,0.004397,0.020256,0.018522,0.009038,...,-0.002972,-0.016259,-0.003040,0.014383,-0.007552,0.002171,-0.014738,0.002990,0.017127,0


In [6]:
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
    dataframe = dataframe.copy()
    labels = dataframe.pop('id')
#     labels = to_categorical(labels)
    ds = tf.data.Dataset.from_tensor_slices((dataframe, labels))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(dataframe))
    ds = ds.batch(batch_size)
    ds = ds.prefetch(batch_size)
    return ds

In [7]:
test_ds = df_to_dataset(dataframe_test, shuffle=False)

### Load model

In [8]:
reloaded_model = tf.keras.models.load_model('mixed_classification_for_masked')

### Predict 

In [9]:
id = dataframe_test['id'].values.tolist()
dataframe_test = dataframe_test.drop('id', axis=1)
dataframe_test.head()

,0,1,2,3,4,5,6,7,8,9,...,2612,2613,2614,2615,2616,2617,2618,2619,2620,2621
0,-0.005811,0.006799,0.006527,0.006695,0.014906,0.011761,0.002441,0.014137,0.001551,0.000994,...,-0.006069,-0.005224,-0.022632,-0.011859,0.019855,-0.008901,-0.008245,-0.014659,0.000927,0.017252
1,-0.000615,0.004171,0.008273,0.014302,0.010149,0.014710,0.005147,0.013600,0.005837,0.008584,...,-0.008443,-0.000316,-0.016601,-0.001300,0.022648,0.003575,0.014947,-0.019859,0.008109,0.021828
2,-0.004168,-0.001410,0.005084,0.008728,0.003481,0.017883,-0.002308,0.017414,0.006853,0.008259,...,-0.006941,-0.011242,-0.016792,-0.007640,0.021043,-0.004024,0.000192,-0.012531,0.004651,0.023042
3,0.000304,0.011040,0.014343,0.013360,0.009079,0.017290,0.014120,0.020558,-0.001548,0.008252,...,-0.011087,-0.012067,-0.018902,-0.004995,0.020847,-0.003581,-0.001372,-0.016296,0.007014,0.029171
4,-0.003455,-0.003399,0.012119,0.010312,0.004979,0.024713,0.004397,0.020256,0.018522,0.009038,...,-0.006413,-0.002972,-0.016259,-0.003040,0.014383,-0.007552,0.002171,-0.014738,0.002990,0.017127


In [10]:
print(np.array(dataframe_test.loc[0].values.flatten()))
print(len(np.array(dataframe_test.loc[0].values.flatten())))
print(np.expand_dims(dataframe_test.loc[0],0))
print(np.expand_dims(dataframe_test.loc[0],0).size)

[-0.00581149  0.00679936  0.00652705 ... -0.01465872  0.00092726
  0.01725205]
2622
[[-0.00581149  0.00679936  0.00652705 ... -0.01465872  0.00092726
   0.01725205]]
2622


In [11]:
result = reloaded_model.predict(np.expand_dims(dataframe_test.loc[0],0))
print(result)
print(np.argsort(-result)) # - is descending order
index_numpy_sorted = np.argsort(-result)

[[1.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 7.1735704e-34
  0.0000000e+00 0.0000000e+00 1.1404924e-34 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00 8.1732893e-31 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00 4.1073844e-34 2.4031038e-25 0.0000000e+00 0.0000000e+00
  4.2810890e-32 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  4.3448116e-38 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  2.6471753e-27 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 2.1400765e-37 0.0000000e+00 0.0000000e+00]]
[[ 0 22 35 16 25  4 21  7 47 30 29 31 32 33 34 37 38 39 40 41 42 43 44 45
  46 36 28 24 26  1  2  3  5  6  8  9 10 11 12 13 14 15 17 18 19 20 23 48
  27 49]]


In [12]:
print(index_numpy_sorted[0][0])

0


### HIT at K and Precision at K

In [13]:
for k in range(1,11):
    arr_temp = []
    arr_temp_2 = []
    for i in range(len(id)):
        result = reloaded_model.predict(np.expand_dims(dataframe_test.loc[i],0))
        index_numpy_sorted = np.argsort(-result)
        booleannaja = False
        for j in range(k):
            if str(id[i]) == str(index_numpy_sorted[0][j]):
                booleannaja = True
        if booleannaja is True:
            arr_temp.append('Correct')
            arr_temp_2.append(1/k)
        else:
            arr_temp.append('Incorrect')
            arr_temp_2.append(0)
    print('-=-=-=-=-=-=' + str(k) + '-=-=-=-=-=-=')
    print(len(arr_temp))
    print(arr_temp.count('Correct'))
    print('percentage: ' + str((arr_temp.count('Correct') * 100) / len(id)))
    print(len(arr_temp_2))
    print(sum(arr_temp_2))
    print(sum(arr_temp_2) / len(id))

-=-=-=-=-=-=1-=-=-=-=-=-=
2087
1781
percentage: 85.3378054623862
2087
1781.0
0.853378054623862
-=-=-=-=-=-=2-=-=-=-=-=-=
2087
1872
percentage: 89.69813128893148
2087
936.0
0.4484906564446574
-=-=-=-=-=-=3-=-=-=-=-=-=
2087
1920
percentage: 91.99808337326306
2087
640.0000000000028
0.30666027791087824
-=-=-=-=-=-=4-=-=-=-=-=-=
2087
1946
percentage: 93.24389075227599
2087
486.5
0.23310972688068998
-=-=-=-=-=-=5-=-=-=-=-=-=
2087
1964
percentage: 94.10637278390034
2087
392.79999999998626
0.18821274556779408
-=-=-=-=-=-=6-=-=-=-=-=-=
2087
1978
percentage: 94.77719214183038
2087
329.6666666666692
0.15796198690305183
-=-=-=-=-=-=7-=-=-=-=-=-=
2087
1982
percentage: 94.96885481552468
2087
283.1428571428569
0.13566979259360656
-=-=-=-=-=-=8-=-=-=-=-=-=
2087
1986
percentage: 95.16051748921898
2087
248.25
0.11895064686152372
-=-=-=-=-=-=9-=-=-=-=-=-=
2087
1992
percentage: 95.44801149976043
2087
221.3333333333386
0.10605334611084744
-=-=-=-=-=-=10-=-=-=-=-=-=
2087
1997
percentage: 95.68758984187829
2